# 👥 Proyecto: Limpieza de Datos Demográficos (Clientes)

## 📋 Descripción
Procesamiento del archivo `customer-details.xlsx`. Se estandarizan formatos y se asegura la calidad de los datos para la unión posterior.

## 📂 Archivos
* **Entrada:** `data/customer-details.xlsx`
* **Salida:** `data/procesados/customer_clean.csv`

## ⚙️ Estructura y Funciones
* **Celda 1:** Configuración y Carga (Excel).
* **Celda 2:** **Estandarización General**
    * Limpieza de títulos (`sl.limpiar_titulos`)
    * Limpieza de textos (`sl.limpiar_texto`) - *Protegiendo el ID*.
* **Celda 3:** **Correcciones Específicas**
    * Renombrado de clave ID (`sl.cambiar_nombres`)
    * Gestión de 'unknown' (`no usamos nada`)
    * Conversión de fechas estándar (`pd.to_datetime`)
* **Celda 4:** **Validación Numérica**
    * Conversión segura a numérico (`pd.to_numeric`)
    * Eliminación de duplicados (`sl.eliminar_duplicados`)
* **Celda 5:** **Cierre**
    * Eliminación de columnas basura (`sl.eliminar_columnas`)
    * Guardado CSV.

In [1]:
# -------------------------------------------------------------------------
# CELDA 1: CARGA DE LIBRERÍAS Y DATOS
# -------------------------------------------------------------------------

# 1. CONFIGURACIÓN
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import sys

# VISUALIZACIÓN: Ver todas las columnas (sin puntos suspensivos)
pd.set_option('display.max_columns', None)

# Conectamos con la carpeta de funciones
sys.path.append(os.path.abspath(".."))

from src import sp_lim as sl
from src import sp_eda as se

# 2. CARGAR DATOS
df_original = pd.read_excel("../data/customer-details.xlsx")

# 3. CREAR COPIA DE TRABAJO
df_customer = df_original.copy()

print(f"✅ Configuración lista. Copia cargada: {df_customer.shape}")
display(df_customer.head(2))

✅ Configuración lista. Copia cargada: (20115, 7)


,Unnamed: 0,Income,Kidhome,Teenhome,Dt_Customer,NumWebVisitsMonth,ID
0,0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079
1,1,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097


In [2]:
# -------------------------------------------------------------------------
# CELDA 2: ESTANDARIZACIÓN GENERAL (Corregida)
# -------------------------------------------------------------------------

# 1. ARREGLAR TÍTULOS DE COLUMNAS
# La columna 'ID' se convertirá automáticamente en 'id'
sl.limpiar_titulos(df_customer)

# 2. ARREGLAR TEXTOS
# Protegemos la columna 'id' (porque limpiar_titulos ya la puso en minúsculas)
# También protegemos 'dt_customer' que parece ser la fecha
columnas_intocables = ['id', 'dt_customer'] 

sl.limpiar_texto(df_customer, columnas_a_ignorar=columnas_intocables)

print("✅ Estandarización completada.")
print("Así se ve ahora (fíjate que ID ahora es 'id'):")
display(df_customer.head(3))

✅ Nombres de columnas estandarizados.
🧹 Limpiando contenido de 0 columnas...
✅ Textos estandarizados (sin ñ, espacios, puntos ni guiones).
✅ Estandarización completada.
Así se ve ahora (fíjate que ID ahora es 'id'):


,unnamed:_0,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id
0,0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079
1,1,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097
2,2,147233,1,1,2012-02-02,5,3f9f49b5-e410-4948-bf6e-f9244f04918b


In [3]:
# -------------------------------------------------------------------------
# CELDA 3: CORRECCIONES ESPECÍFICAS
# -------------------------------------------------------------------------

# 1. CAMBIAR EL NOMBRE DE id a ID
sl.cambiar_nombres(df_customer, {'id': 'ID'})

# 2. GESTIONAR VALORES 'UNKNOWN'
# este rchivo no tiene valores de texto 'unknown'

# 3. ARREGLAR LA FECHA
# Usamos conversión directa (formato estándar)
df_customer['dt_customer'] = pd.to_datetime(df_customer['dt_customer'])

print("✅ Correcciones aplicadas.")
print(df_customer[['ID', 'dt_customer']].dtypes) # Para comprobar
display(df_customer.head(3))

✅ Se han renombrado las columnas: ['id']
✅ Correcciones aplicadas.
ID                     object
dt_customer    datetime64[ns]
dtype: object


,unnamed:_0,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,ID
0,0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079
1,1,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097
2,2,147233,1,1,2012-02-02,5,3f9f49b5-e410-4948-bf6e-f9244f04918b


In [4]:
# -------------------------------------------------------------------------
# CELDA 4: NÚMEROS Y LIMPIEZA FINAL (CLIENTES)
# -------------------------------------------------------------------------

# 1. ARREGLAR NÚMEROS
# No usamos 'sl.limpiar_numeros' porque aquí no hay comas que quitar.
# Usamos conversión directa para asegurar que todo sea numérico.
if 'income' in df_customer.columns:
    df_customer['income'] = pd.to_numeric(df_customer['income'], errors='coerce')

# 2. ELIMINAR DUPLICADOS
# Esto siempre es obligatorio
sl.eliminar_duplicados(df_customer)

print("\n--- INFO FINAL ---")
df_customer.info() # Verifica que 'income' sea float64

✅ No se encontraron duplicados.

--- INFO FINAL ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20115 entries, 0 to 20114
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   unnamed:_0         20115 non-null  int64         
 1   income             20115 non-null  int64         
 2   kidhome            20115 non-null  int64         
 3   teenhome           20115 non-null  int64         
 4   dt_customer        20115 non-null  datetime64[ns]
 5   numwebvisitsmonth  20115 non-null  int64         
 6   ID                 20115 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.1+ MB


In [5]:
# -------------------------------------------------------------------------
# CELDA 5: LIMPIEZA FINAL Y GUARDADO (CLIENTES)
# -------------------------------------------------------------------------

# 1. BORRAR COLUMNAS SOBRANTES
# Si existiera la columna basura 'unnamed:_0', la quitamos.
sl.eliminar_columnas(df_customer, ['unnamed:_0'])

# 2. GUARDADO EN CARPETA 'PROCESADOS'
# La ruta es: Atrás (..) -> entra en data -> entra en procesados -> nombre del archivo
ruta_guardado = '../data/procesados/customer_clean.csv'

df_customer.to_csv(ruta_guardado, index=False)

print(f"✅ ¡TRABAJO TERMINADO!")
print(f"💾 Archivo limpio guardado en: {ruta_guardado}")

# 3. VERIFICACIÓN FINAL (VISTA DE LA TABLA)
print("\n--- 🔍 VISTA PREVIA DEL ARCHIVO FINAL ---")
display(df_customer.head(10))

print("\n--- INFORMACIÓN TÉCNICA ---")
df_customer.info()

🗑️ Columnas eliminadas: ['unnamed:_0']
✅ ¡TRABAJO TERMINADO!
💾 Archivo limpio guardado en: ../data/procesados/customer_clean.csv

--- 🔍 VISTA PREVIA DEL ARCHIVO FINAL ---


,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,ID
0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079
1,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097
2,147233,1,1,2012-02-02,5,3f9f49b5-e410-4948-bf6e-f9244f04918b
3,121393,1,2,2012-12-21,29,9991fafb-4447-451a-8be2-b0df6098d13e
4,63164,1,2,2012-06-20,20,eca60b76-70b6-4077-80ba-bc52e8ebb0eb
5,143854,0,1,2012-01-24,26,d63ede72-0b6d-45b1-8872-385ac6897f65
6,76439,1,0,2012-08-15,3,5e3483e5-236d-437d-8351-541f9d09b9dd
7,159686,1,1,2012-12-10,21,87fdc08b-30ae-4dab-803f-561ecdf27ff0
8,179933,0,0,2012-08-22,18,87b79988-2be5-419d-88f4-56655852c565
9,77504,1,0,2012-02-09,18,ea6b7d04-9271-4c0a-a01f-07795d164aba



--- INFORMACIÓN TÉCNICA ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20115 entries, 0 to 20114
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   income             20115 non-null  int64         
 1   kidhome            20115 non-null  int64         
 2   teenhome           20115 non-null  int64         
 3   dt_customer        20115 non-null  datetime64[ns]
 4   numwebvisitsmonth  20115 non-null  int64         
 5   ID                 20115 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 943.0+ KB
